## 1. Preprocess NHTS data

In [2]:
import pandas as pd
import numpy as np
import os
import re
import warnings
from tqdm.auto import tqdm
# 경고 메시지 무시 설정
warnings.filterwarnings('ignore')

## Code
### 1) Code for preprocessing NHTS data

In [3]:
def organize_columns(df, print_progress = True):
    trippub_re = df.copy()
    
    # columns arrangement
    columns = ['HOUSEID', 'PERSONID', 'HHSTFIPS','R_AGE', 'R_AGE_IMP', 'TRIPPURP', 'WHYTRP1S', 'WHYTRP90', 'WHYFROM', 'WHYTO', 'TRPMILES', 'DWELTIME','STRTTIME', 'ENDTIME', 'TRPTRANS','TRAVDAY', 'TDAYDATE']
    trippub_re = trippub_re[columns]
    
    # 0) TRPPURP_column
    tqdm.pandas(desc="0) mapping TRPPURP")
    
    # 새로운 컬럼 TRPPURP_new 생성
    def map_purpose(row):
        if row['WHYTO'] in [1, 2]:
            return 'Home'
        elif row['WHYTO'] in [3, 4]:
            return 'Work'
        elif row['WHYTO'] in [6, 8, 9, 10, 19]:
            return 'S_d_r'
        elif row['WHYTO'] == 11:
            return 'D_shop'
        elif row['WHYTO'] == 13:
            return 'Meals'
        elif row['WHYTO'] == 17:
            return 'V_fr_rel'
        elif row['WHYTO'] in [15, 16]:
            return 'Rec_lei'
        elif row['WHYTO'] in [12, 14, 18]:
            return 'Serv_trip'
        elif row['WHYTO'] in [5, 97]:
            return 'Others'
        else:
            return None
        
        
    if print_progress == True:
        trippub_re['TRPPURP_new'] = trippub_re.progress_apply(map_purpose, axis=1)
    else:
        trippub_re['TRPPURP_new'] = trippub_re.apply(map_purpose, axis=1)
    
    
    
    # 새로운 컬럼 TRPTRANS 생성
    
    # 1) TRPTRANS column
    tqdm.pandas(desc="1) mapping TRPTRANS")
    
    def map_mode(row):
        if row['TRPTRANS'] == 1:
            return 'Walk'
        elif row['TRPTRANS'] == 2:
            return 'Bicy'
        elif row['TRPTRANS'] in [3,4,5,6,8,9,10,18]:
            return 'Car'
        elif row['TRPTRANS'] in [10,11,12,13,14,16]:
            return 'PTrans'
        elif row['TRPTRANS'] == 19:
            return 'Air'
        else:
            return None
    
    if print_progress == True:    
        trippub_re['TRPTRANS_new'] = trippub_re.progress_apply(map_mode, axis=1)
    else:
        trippub_re['TRPTRANS_new'] = trippub_re.apply(map_mode, axis=1)
    
    
    
    # 새로운 컬럼 TRAVDAY 생성
    
    # 2) TRAVDAY column
    tqdm.pandas(desc="2) mapping TRAVDAY")
    
    def map_week(row):
        if row['TRAVDAY'] in [1,7]:
            return 'Weekend'
        elif row['TRAVDAY'] in [2,3,4,5,6]:
            return 'Weekday'
        else:
            return None
    
    
    if print_progress == True:    
        trippub_re['TRAVDAY_new'] = trippub_re.progress_apply(map_week, axis=1)
    else:
        trippub_re['TRAVDAY_new'] = trippub_re.apply(map_week, axis=1)
    
    
    # 새로운 컬럼 R_AGE_new 생성
    
    # 3) R_AGE_new column
    tqdm.pandas(desc="3) mapping R_AGE_new")
    
    def reclassify_age(age):
        if age < 10:
            return 'Child'
        elif 10 <= age < 20:
            return 'Teen'
        elif 20 <= age < 40:
            return 'Adult'
        elif 40 <= age < 60:
            return 'MidAdult'
        else:
            return 'Seniors'

    if print_progress == True:    
        trippub_re['R_AGE_new'] = trippub_re['R_AGE_IMP'].progress_apply(reclassify_age)
    else:
        trippub_re['R_AGE_new'] = trippub_re['R_AGE_IMP'].apply(reclassify_age)
    
    
    # 새로운 컬럼 ID
    
    
    if print_progress == True:   
        print()
        print('4) Now, Other columns...')
    
    # 정수형을 문자열로 변환
    trippub_re['HOUSEID'] = trippub_re['HOUSEID'].astype(str)
    trippub_re['PERSONID'] = trippub_re['PERSONID'].astype(str)
    # 두 문자열 컬럼을 합치고 새로운 컬럼에 저장
    trippub_re['PERSONID_new'] = trippub_re['HOUSEID'] + trippub_re['PERSONID']
    
    
    # 값이 없는 행 수 세기 - Null값
    num_empty_rows = trippub_re['TRPPURP_new'].isnull().sum()
#     print('값이 없는 행의 수: ', num_empty_rows)

    # 값이 없는 행 제거
    trippub_re.dropna(subset=['TRPPURP_new'], inplace=True)
    trippub_re.dropna(subset=['TRPTRANS_new'], inplace=True)
    trippub_re.dropna(subset=['R_AGE_IMP'], inplace=True)
    trippub_re.dropna(subset=['TRAVDAY_new'], inplace=True)
    
    
    if print_progress == True:   
        print()
        print('... done!')
        
    # Filtering Columns
    selected_columns = ['HOUSEID', 'PERSONID', 'PERSONID_new', 'HHSTFIPS', 'R_AGE_IMP', 'R_AGE_new', 'WHYFROM', 'WHYTO', 'TRPMILES', 'DWELTIME',
                        'STRTTIME', 'ENDTIME', 'TRAVDAY', 'TRAVDAY_new', 'TDAYDATE', 'TRPPURP_new', 'TRPTRANS_new']
    
    trippub_re = trippub_re[selected_columns]
    
    
    return trippub_re

In [4]:
def preprocess_NHTS(df, print_progress = True):
    
    # 1) DWELTIME 컬럼에서 0보다 작은 값을 0으로 변경
    trip_total = df.copy()
    
    if print_progress == True:
        print('Start preprocessing... NHTS data...')
        
    trip_total['DWELTIME'] = trip_total['DWELTIME'].apply(lambda x: 0 if x < 0 else x)
    
     # TRAVTIME 계산을 위해 시간과 분을 따로 추출
    trip_total['ARRIVAL_hour'] = trip_total['ENDTIME'] // 100
    trip_total['ARRIVAL_minute'] = trip_total['ENDTIME'] % 100
    trip_total['STRTTIME_hour'] = trip_total['STRTTIME'] // 100
    trip_total['STRTTIME_minute'] = trip_total['STRTTIME'] % 100
    
    # 분 단위로 변환
    trip_total['ARRIVAL_in_minutes'] = trip_total['ARRIVAL_hour'] * 60 + trip_total['ARRIVAL_minute']
    trip_total['STRTTIME_in_minutes'] = trip_total['STRTTIME_hour'] * 60 + trip_total['STRTTIME_minute']
    
    # end time
    trip_total['ENDTIME_minute'] = trip_total['ARRIVAL_in_minutes'] + trip_total['DWELTIME']
    
    # 2) TRAVTIME 계산 (분 단위)
    trip_total['TRAVTIME'] = trip_total['ARRIVAL_in_minutes'] - trip_total['STRTTIME_in_minutes']
    
    # 새벽이 넘어간 경우
    trip_total['TRAVTIME'] = trip_total['TRAVTIME'].apply(lambda x: x if x >= 0 else x + 2400)
    
   
    # DWELTIME의 합이 0인 PERSONID_new와 TRAVDAY_new 조합 행 제거
    sum_dweltime = trip_total.groupby(['PERSONID_new', 'TRAVDAY_new'])['DWELTIME'].sum().reset_index()
    valid_ids = sum_dweltime[sum_dweltime['DWELTIME'] != 0][['PERSONID_new', 'TRAVDAY_new']]
    trip_total = pd.merge(trip_total, valid_ids, on=['PERSONID_new', 'TRAVDAY_new'])
    
    # 컬럼 이름 변경
    trip_total = trip_total.rename(columns={
        'PERSONID_new': 'uniqID',
        'R_AGE_IMP': 'age_Group',
        'TRAVDAY_new': 'Day_Type',
        'TRPPURP_new': 'Trip_pur',
        'DWELTIME': 'Dwell_T_min',
        'TRAVTIME': 'Trip_T_min',
        'STRTTIME': 'sta_T_hms',
        'STRTTIME_in_minutes': 'sta_T_min',
        'ENDTIME': 'arr_T_hms',
        'ARRIVAL_in_minutes': 'arr_T_min',
        'ENDTIME_minute' : 'end_T_min'
    })
    
    trip_total = trip_total[['uniqID', 'age_Group', 'Day_Type', 'Trip_pur', 'sta_T_hms', 'arr_T_hms', 'Dwell_T_min', 'Trip_T_min', 'sta_T_min', 'arr_T_min', 'end_T_min']]
      # 1) Day_Type, uniqID, sta_T_min순으로 ascending order로 정렬
    trip_total = trip_total.sort_values(['Day_Type', 'uniqID', 'sta_T_min'])
    # display(trip_total)

    
    # 2) 만약 Day_Type별 uniqID로 묶어서 ascending order로 보았을 때 각 그룹의 Trip_pur이 'Home'으로 시작하는데 같은 그룹안 다음줄에 또 'Home'이 있으면 첫번째 'Home'이 있는 줄은 지워주기
    tqdm.pandas(desc="0) remove duplicate home")
    
    def remove_duplicate_home(group):
        if len(group) > 1:
            if group.iloc[0]['Trip_pur'] == 'Home' and group.iloc[1]['Trip_pur'] == 'Home':
                return group.iloc[1:]
        return group

    # 기존에 정렬하는 부분
    trip_total = trip_total.sort_values(['Day_Type', 'uniqID', 'sta_T_min'])

    # 첫 번째 'Home' 삭제 조건을 적용
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(remove_duplicate_home).reset_index(drop=True)
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(remove_duplicate_home).reset_index(drop=True)

    # display(trip_total[trip_total['uniqID'] ==300010541])

    # 3) Day_Type별 uniqID로 묶어서 ascending order로 보았을 때 각 그룹의 Trip_pur이 'Home'으로 시작하지 않는다면 Home으로 시작해야 함.
    tqdm.pandas(desc="1) Start trip from home")
    
    
    def add_home_if_needed(group):
        
        # sta_T_min이 0인 경우 그룹을 삭제
        if group.iloc[0]['sta_T_min'] == 0:
            return None
        
        # 첫번째 행이 'Home'으로 시작하지 않는 경우 새로운 행 추가
        if group.iloc[0]['Trip_pur'] != 'Home':
            new_row = group.iloc[0].copy()
            new_row['Trip_pur'] = 'Home'
            new_row['Dwell_T_min'] = group.iloc[0]['sta_T_min']
            new_row['Trip_T_min'] = 0
            new_row['sta_T_min'] = 0
            new_row['arr_T_min'] = 0
            new_row['end_T_min'] = new_row['Dwell_T_min']
            group = pd.concat([pd.DataFrame([new_row]), group]).reset_index(drop=True)
        return group
    
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(add_home_if_needed).reset_index(drop=True).dropna()
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(add_home_if_needed).reset_index(drop=True).dropna()
    
    

    # 4) 만약 Day_Type별 uniqID로 묶어서 ascending order로 보았을 때 Trip_pur이 'Home'으로 시작한다 해도 sta_T_min이 0이 아니면 바꿔줘야 함.
    tqdm.pandas(desc="2) Adjusting initial Home Time")

    def adjust_home_time(group):
        first_row = group.iloc[0]
        if first_row['Trip_pur'] == 'Home':
            # print(first_row['uniqID'])

            # display(group)
            next_row_idx = first_row.name + 1
            # print(next_row_idx)
            if next_row_idx in group.index:
                next_row = group.loc[next_row_idx]

                group.at[first_row.name, 'sta_T_min'] = 0
                group.at[first_row.name, 'arr_T_min'] = 0
                group.at[first_row.name, 'end_T_min'] = next_row['sta_T_min']
                group.at[first_row.name, 'Dwell_T_min'] = group.at[first_row.name, 'end_T_min']
                group.at[first_row.name, 'Trip_T_min'] = 0
            else:
                group.at[first_row.name, 'sta_T_min'] = 0
                group.at[first_row.name, 'arr_T_min'] = 0
                group.at[first_row.name, 'end_T_min'] = 1440
                group.at[first_row.name, 'Dwell_T_min'] = 1440
                group.at[first_row.name, 'Trip_T_min'] = 0
        return group

    
    # Day_Type와 uniqID로 묶어서 Home 시간 조정
    
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(adjust_home_time).reset_index(drop=True)
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(adjust_home_time).reset_index(drop=True)
    

    # 5) endtime이 24시간이 넘어간 경우 지우기 
    trip_total = trip_total[trip_total['sta_T_min'] < trip_total['end_T_min']]
    
    # 6) Dwell_T_min이 0인 group들 정리하기
    tqdm.pandas(desc="3) Addressing Dwell Time")  # tqdm의 pandas 확장을 활성화

    def filter_invalid_groups(group):
#         global deleted_rows_counter
        dwell_zero_count = sum(group['Dwell_T_min'] == 0)
        last_row = group.iloc[-1]

        # 조건에 따라 그룹을 삭제
        if dwell_zero_count >= 2:
#             deleted_rows_counter += len(group)
            return None
        if dwell_zero_count >= 1 and last_row['Trip_pur'] != 'Home':
#             deleted_rows_counter += len(group)
            return None
        if last_row['Trip_pur'] != 'Home' and last_row['Dwell_T_min'] == 0:
#             deleted_rows_counter += len(group)
            return None

        return group

    # Day_Type와 uniqID로 그룹을 나누고 필터링
    
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(filter_invalid_groups).reset_index(drop=True).dropna()
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(filter_invalid_groups).reset_index(drop=True).dropna()


    # 7) Day_Type별 uniqID로 묶어서 ascending order로 보았을 때 각 그룹의 Trip_pur이 'Home'으로 끝나지 않는다면 Home으로 끝나야 함.
    tqdm.pandas(desc="4) Add Home as a last travel")  # tqdm의 pandas 확장을 활성화
    
    def add_or_remove_home_at_end(group):
        # 새로운 행 추가 부분
#         global deleted_rows_counter
        
        if group.iloc[-1]['Trip_pur'] != 'Home':
            last_row = group.iloc[-1]
            new_row = last_row.copy()
            new_row['Trip_pur'] = 'Home'
            new_row['sta_T_min'] = last_row['end_T_min']

            trip_t_min_mean = int(group[group['Trip_T_min'] != 0]['Trip_T_min'].mean())

            new_row['arr_T_min'] = trip_t_min_mean + new_row['sta_T_min']

            # arr_T_min이 1440을 넘어가면 해당 그룹 삭제
            if new_row['arr_T_min'] > 1440:
#                 deleted_rows_counter += len(group)
                return None

            new_row['end_T_min'] = 1440
            new_row['Trip_T_min'] = new_row['arr_T_min'] - new_row['sta_T_min']
            new_row['Dwell_T_min'] = 1440 - new_row['arr_T_min']

            # sta_T_hms와 end_T_hms 업데이트
            new_row['sta_T_hms'] = (new_row['sta_T_min'] // 60) * 100 + (new_row['sta_T_min'] % 60)
            new_row['arr_T_hms'] = (new_row['arr_T_min'] // 60) * 100 + (new_row['arr_T_min'] % 60)

            # 새로운 행을 기존 그룹에 추가
            group = pd.concat([group, pd.DataFrame([new_row])]).reset_index(drop=True)
            # display(group)
        return group

    # Apply the function to each group
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(add_or_remove_home_at_end).reset_index(drop=True).dropna()
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(add_or_remove_home_at_end).reset_index(drop=True).dropna()
    
    
    # 8) 마지막과 마지막 전 줄의 Trip_pur이 'Home'인 경우 마지막 전 줄을 지우고 column 정리
    tqdm.pandas(desc="5) Remove duplicate last Home and Adjust")  # tqdm의 pandas 확장을 활성화

    def remove_last_home(group):
        group.reset_index(inplace = True, drop=True)
        last_idx = len(group) - 1
        if last_idx > 0: # 그룹에 최소 두 개의 행이 있는 경우에만 검사
            if group.iloc[last_idx]['Trip_pur'] == 'Home' and group.iloc[last_idx - 1]['Trip_pur'] == 'Home':
                # print(group['uniqID'])
                group = group.iloc[:-1]  # 마지막 줄을 지움
                
                # 마지막 줄을 지운 후 새로운 마지막 줄을 수정
                group.reset_index(inplace = True, drop=True)
                new_last_idx = len(group) - 1
                group.at[new_last_idx, 'end_T_min'] = 1440
                group.at[new_last_idx, 'Dwell_T_min'] = 1440 - group.at[new_last_idx, 'arr_T_min']

                # sta_T_hms와 arr_T_hms 형식을 바꿈
                group.at[new_last_idx, 'sta_T_hms'] = (group.at[new_last_idx, 'sta_T_min'] // 60) * 100 + (group.at[new_last_idx, 'sta_T_min'] % 60)
                group.at[new_last_idx, 'arr_T_hms'] = (group.at[new_last_idx, 'arr_T_min'] // 60) * 100 + (group.at[new_last_idx, 'arr_T_min'] % 60)

                # display(group)
        return group
    
    # Apply the function to each group
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(remove_last_home).reset_index(drop=True).dropna()
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(remove_last_home).reset_index(drop=True).dropna()
     
    
    # 9) Home으로 끝나는 그룹의 시간 정리
    tqdm.pandas(desc="6) Adjusting Last Home Time")  # tqdm의 pandas 확장을 활성화
    
    def adjust_home_time(group):
        group.reset_index(inplace = True, drop=True)
        
        last_idx = len(group) - 1
        if last_idx >= 0:  # 그룹에 최소 한 개의 행이 있는 경우에만 검사
            if group.iloc[last_idx]['Trip_pur'] == 'Home':
                # 마지막 줄의 end_T_min을 1440으로 설정
                group.at[last_idx, 'end_T_min'] = 1440
                # 마지막 줄의 Dwell_T_min을 1440 - arr_T_min으로 설정
                group.at[last_idx, 'Dwell_T_min'] = 1440 - group.at[last_idx, 'arr_T_min']

                # sta_T_hms와 arr_T_hms 형식을 바꿈

                # sta_T_hms와 arr_T_hms 형식을 바꿈
                group.at[last_idx, 'sta_T_hms'] = (group.at[last_idx, 'sta_T_min'] // 60) * 100 + (group.at[last_idx, 'sta_T_min'] % 60)
                group.at[last_idx, 'arr_T_hms'] = (group.at[last_idx, 'arr_T_min'] // 60) * 100 + (group.at[last_idx, 'arr_T_min'] % 60)

        # display(group)
        return group
    
    # Apply the function to each group
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(adjust_home_time).reset_index(drop=True).dropna()
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(adjust_home_time).reset_index(drop=True).dropna()

    
    # 10) Home이 첫, 끝 trip을 제외하고 중간에 여러개가 연속으로 있을 경우, 하나의 Home트립으로 바꿔줌
    tqdm.pandas(desc="7) merge consecutive home")  # tqdm의 pandas 확장을 활성화
    
    # Assuming repaired_orig_data is your DataFrame
    def merge_consecutive_home_groups(group):
        consecutive_home_groups = []
        current_group = []

        for idx, row in group.iterrows():
            if row['Trip_pur'] == 'Home':
                current_group.append(idx)
            else:
                if current_group:
                    consecutive_home_groups.append(current_group.copy())
                    current_group = []

        if current_group:
            consecutive_home_groups.append(current_group)

        # Merge consecutive home groups
        for home_group in consecutive_home_groups:
            if len(home_group) > 1:
                total_dwell_time = group.loc[home_group, 'Dwell_T_min'].sum()
                total_trip_time = group.loc[home_group, 'Trip_T_min'].sum()

                # Update the end time of the first row with the end time of the last row in the group
                group.at[home_group[0], 'end_T_min'] = group.at[home_group[-1], 'end_T_min']
            
                # Update the first row in the consecutive home group
                group.at[home_group[0], 'Dwell_T_min'] = group.at[home_group[0], 'end_T_min'] - group.at[home_group[0], 'arr_T_min']

                # Drop the rows in the consecutive home group except for the first one
                group = group.drop(home_group[1:])

        # Reset the index
        group = group.reset_index(drop=True)

        # Update sta_T_hms and arr_T_hms
        group['sta_T_hms'] = (group['sta_T_min'] // 60) * 100 + (group['sta_T_min'] % 60)
        group['arr_T_hms'] = (group['arr_T_min'] // 60) * 100 + (group['arr_T_min'] % 60)

        # display(group)
        return group

    # Apply the function to each group
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(merge_consecutive_home_groups).reset_index(drop=True).dropna()
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(merge_consecutive_home_groups).reset_index(drop=True).dropna()

   
    # 11) 만약 Day_Type별 uniqID로 묶어서 ascending order로 보았을 때 Trip_pur이 'Home'으로 시작하지 않거나 끝나지 않는 Day_Type별 uniqID는 지움 - 없음
    
    # 12) 이전의 end_T_min이 다음 줄의 sta_T_min과 일치하지 않으면 일치하도록
    tqdm.pandas(desc="8) align 'start time' with next 'end time'")  # tqdm의 pandas 확장을 활성화
    
    def align_end_time_with_next_start_time(group):
        for idx in range(len(group) - 1):
            current_row = group.iloc[idx]
            next_row = group.iloc[idx + 1]

            if current_row['end_T_min'] != next_row['sta_T_min']:
                # Update end_T_min and Dwell_T_min of the current row
                group.at[current_row.name, 'end_T_min'] = next_row['sta_T_min']
                group.at[current_row.name, 'Dwell_T_min'] = next_row['sta_T_min'] - current_row['arr_T_min']

        return group

    # Apply the function to each group
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(align_end_time_with_next_start_time).reset_index(drop=True)
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(align_end_time_with_next_start_time).reset_index(drop=True)
    
    
    # 13) 그룹의 첫, 마지막 컬럼값이 Home이 아니거나 시간이 make sense하지 않으면 지우기
    tqdm.pandas(desc="9) filter Home as a start and end pur")  # tqdm의 pandas 확장을 활성화
    
    def filter_group(group):
#         global deleted_rows_counter
        
        group.reset_index(inplace=True, drop=True)
        last_idx = len(group) - 1
        first_idx = 0

        if last_idx >= 0:  # 그룹에 최소 한 개의 행이 있는 경우에만 검사
            first_condition = group.at[first_idx, 'sta_T_min'] != 0
            last_condition = group.at[last_idx, 'end_T_min'] != 1440
            trip_pur_condition = group.at[last_idx, 'Trip_pur'] != 'Home'
            
            if first_condition or last_condition or trip_pur_condition:
#                 deleted_rows_counter += len(group)
                return None  # 조건에 맞지 않는 그룹은 제거
        return group

    # Apply the function to each group and filter out None values
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(filter_group).reset_index(drop=True).dropna()
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(filter_group).reset_index(drop=True).dropna()
    
    

#   14) sta_T_hms, arr_T_hms을 sta_T_min arr_T_min에 맞추어 바꾸기
    tqdm.pandas(desc="10) refine start, arr, end time with hms format")  # tqdm의 pandas 확장을 활성화
        
    def hms_format(group):        
        group['sta_T_hms'] = (group['sta_T_min'] // 60) * 100 + (group['sta_T_min'] % 60)
        group['arr_T_hms'] = (group['arr_T_min'] // 60) * 100 + (group['arr_T_min'] % 60)
        
        # Add end_T_min
        group['end_T_hms'] = (group['end_T_min'] // 60) * 100 + (group['end_T_min'] % 60)
        
        return group
        
    
    # Apply the function to each group and filter out None values
    trip_total['end_T_hms'] = None
    
    if print_progress == True:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).progress_apply(hms_format).reset_index(drop=True).dropna()
    else:
        trip_total = trip_total.groupby(['Day_Type', 'uniqID']).apply(hms_format).reset_index(drop=True).dropna()


#   15) Reclassifying ages
    tqdm.pandas(desc="11) Reclassifying age variables")  # tqdm의 pandas 확장을 활성화        
    
    def reclassify_age(age):
        if age < 10:
            return 'Child'
        elif 10 <= age < 20:
            return 'Teen'
        elif 20 <= age < 40:
            return 'Adult'
        elif 40 <= age < 60:
            return 'MidAdult'
        else:
            return 'Seniors'

        
    if print_progress == True:
        trip_total['age_class'] = trip_total['age_Group'].progress_apply(reclassify_age)
    else:
        trip_total['age_class'] = trip_total['age_Group'].apply(reclassify_age)
    
    if "Unnamed: 0" in trip_total.columns:
        trip_total.drop("Unnamed: 0", axis=1, inplace=True)
    
    trip_total.rename(columns={"age_Group": "age"}, inplace=True)
    
    
    # display(trip_total.head())
    
    # 지운 행의 수를 출력
#     print("Deleted Rows: ", deleted_rows_counter)
    
    
    return trip_total[['uniqID', 'age', 'age_class', 'Day_Type', 'Trip_pur', 'sta_T_hms', 'arr_T_hms', 'end_T_hms', 'Dwell_T_min', 'Trip_T_min', 'sta_T_min', 'arr_T_min', 'end_T_min']]


### 2) Preprocessing for trip mode

In [15]:
# 새로운 컬럼 TRPPURP_new 생성
def preprocess_NHTS_tripMode(df, print_progress = True):
    trippub_total = df.copy()
    
    tqdm.pandas(desc="0) Mapping trippurpose") 
    
    # Maps WHYFROM values to more generalized trip purpose categories.
    def map_purpose(row):
        if row['WHYFROM'] in [1, 2]:
            return 'Home'
        elif row['WHYFROM'] in [3, 4]:
            return 'Work'
        elif row['WHYFROM'] in [6, 8, 9, 10, 19]:
            return 'S_d_r'
        elif row['WHYFROM'] == 11:
            return 'D_shop'
        elif row['WHYFROM'] == 13:
            return 'Meals'
        elif row['WHYFROM'] == 17:
            return 'V_fr_rel'
        elif row['WHYFROM'] in [15, 16]:
            return 'Rec_lei'
        elif row['WHYFROM'] in [12, 14, 18]:
            return 'Serv_trip'
        elif row['WHYFROM'] in [5, 97]:
            return 'Others'
        else:
            return None
        
    # Apply the mapping function to create a new column for the mapped trip purposes.
    trippub_total['TRPFROM_new'] = trippub_total.progress_apply(map_purpose, axis=1)
    
    # Removes an unnecessary column if present.
    if "Unnamed: 0" in trippub_total.columns:
        trippub_total.drop('Unnamed: 0', axis=1, inplace=True)
    
    # replace name of R_AGE_new with age_class
    trippub_total.rename(columns={'R_AGE_new': 'age_class'}, inplace=True)
    
    if print_progress == True:
        print('1) Compute probability of trip mode choice by age and trip purpose ... ')
    
    # trippub_total을 aggregate
    trippub_total = trippub_total.drop_duplicates(subset=['PERSONID_new', 'age_class', 'TRPPURP_new', 'TRPTRANS_new'])
    trippub_total = trippub_total[['PERSONID_new', 'age_class', 'TRPPURP_new', 'TRPTRANS_new']]

    trippub_total.rename(columns={'PERSONID_new': 'uniqID'}, inplace=True)
    trippub_total.rename(columns={'TRPPURP_new': 'Trip_pur'}, inplace=True)
    trippub_total.rename(columns={'TRPTRANS_new': 'Trip_mode'}, inplace=True)
    trippub_total.reset_index(inplace = True, drop = True)
    
        # Count the number of each Trip_mode per age_class and Trip_pur
    trippub_total = trippub_total[trippub_total['Trip_mode'] != 'Air'] 

    count_df_total = trippub_total.groupby(['age_class', 'Trip_pur', 'Trip_mode']).size().reset_index(name='nominator')

    # Count the total number of records per age_class and Trip_pur
    total_df_total = trippub_total.groupby(['age_class', 'Trip_pur']).size().reset_index(name='denominator')

    # Merge the two dataframes on age_class and Trip_pur
    result_total = pd.merge(count_df_total, total_df_total, on=['age_class', 'Trip_pur'])

    # Calculate the probability
    result_total['Trip_modeP'] = result_total['nominator'] / result_total['denominator']
    result_total
    
    return result_total

## Execution
### 1) Mapping Columns

In [3]:
# https://nhts.ornl.gov/
path = 'E:/data/Travel_survey/NHTS/2017/csv/'

trippub = pd.read_csv(path + 'trippub.csv')

In [4]:
trippub.head()

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,ENDTIME,TRVLCMIN,TRPMILES,TRPTRANS,TRPACCMP,TRPHHACC,...,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN
0,30000007,1,1,1000,1015,15,5.244,3,0,0,...,20,750,300,50,750,300,350,30,300,300
1,30000007,1,2,1510,1530,20,5.149,3,0,0,...,30,300,300,50,1500,750,750,20,750,300
2,30000007,2,1,700,900,120,84.004,6,0,0,...,40,1500,750,50,1500,750,750,20,750,300
3,30000007,2,2,1800,2030,150,81.628,6,0,0,...,20,750,300,40,1500,750,750,40,1500,750
4,30000007,3,1,845,900,15,2.250,3,0,0,...,20,750,300,50,750,300,350,60,750,300


In [105]:
trippub_re = organize_columns(trippub[0:100], print_progress = True)

0) mapping TRPPURP:   0%|          | 0/100 [00:00<?, ?it/s]

1) mapping TRPTRANS:   0%|          | 0/100 [00:00<?, ?it/s]

2) mapping TRAVDAY:   0%|          | 0/100 [00:00<?, ?it/s]

3) mapping R_AGE_new:   0%|          | 0/100 [00:00<?, ?it/s]


4) Now, Other columns...

... done!


In [67]:
trippub_re.head()

,HOUSEID,PERSONID,HHSTFIPS,R_AGE_IMP,WHYFROM,WHYTO,TRPMILES,DWELTIME,STRTTIME,ENDTIME,TRAVDAY,TDAYDATE,TRPPURP_new,TRPTRANS_new,R_AGE_new,TRAVDAY_new,PERSONID_new
0,30000007,1,37,67,1,19,5.244,295,1000,1015,2,201608,S_d_r,Car,Seniors,Weekday,300000071
1,30000007,1,37,67,19,1,5.149,-9,1510,1530,2,201608,Home,Car,Seniors,Weekday,300000071
2,30000007,2,37,66,3,1,84.004,540,700,900,2,201608,Home,Car,Seniors,Weekday,300000072
3,30000007,2,37,66,1,3,81.628,-9,1800,2030,2,201608,Work,Car,Seniors,Weekday,300000072
4,30000007,3,37,28,1,8,2.250,330,845,900,2,201608,S_d_r,Car,Adult,Weekday,300000073


In [ ]:
savePath = 'E:/data/Chapter_3_data/Analysis/1_Individual_travel_schedule_generator/NHTS_data/'
# trippub_re.to_csv(savePath + 'trippub_total.csv')


### 2) Preprocessing of NHTS data

In [15]:
savePath = 'E:/data/Travel_survey/NHTS/NHTS_data/'

trippub_total = pd.read_csv(savePath + 'trippub_total.csv')
trippub_wiscon = pd.read_csv(savePath + 'trippub_wiscon.csv')

In [106]:
preprocessed_NHTS = preprocess_NHTS(trippub_total[0:101], print_progress = True)

Start preprocessing... NHTS data...


0) remove duplicate home:   0%|          | 0/26 [00:00<?, ?it/s]

1) Start trip from home:   0%|          | 0/26 [00:00<?, ?it/s]

2) Adjusting initial Home Time:   0%|          | 0/25 [00:00<?, ?it/s]

3) Addressing Dwell Time:   0%|          | 0/25 [00:00<?, ?it/s]

...

7) merge consecutive home:   0%|          | 0/23 [00:00<?, ?it/s]

8) align 'start time' with next 'end time':   0%|          | 0/23 [00:00<?, ?it/s]

9) filter Home as a start and end pur:   0%|          | 0/23 [00:00<?, ?it/s]

10) refine start, arr, end time with hms format:   0%|          | 0/23 [00:00<?, ?it/s]

11) Reclassifying age variables:   0%|          | 0/111 [00:00<?, ?it/s]

In [66]:
preprocessed_NHTS.head()

,uniqID,age,age_class,Day_Type,Trip_pur,sta_T_hms,arr_T_hms,end_T_hms,Dwell_T_min,Trip_T_min,sta_T_min,arr_T_min,end_T_min
0,300000071,67,Seniors,Weekday,Home,0,0,1000,600,0,0,0,600
1,300000071,67,Seniors,Weekday,S_d_r,1000,1015,1510,295,15,600,615,910
2,300000071,67,Seniors,Weekday,Home,1510,1530,2400,510,20,910,930,1440
3,300000073,28,Adult,Weekday,Home,0,0,845,525,0,0,0,525
4,300000073,28,Adult,Weekday,S_d_r,845,900,1430,330,15,525,540,870


In [ ]:
# savePath = 'E:/data/Chapter_3_data/Analysis/1_Preprocessed_NHTS_data/'
# preprocessed_NHTS.to_csv(savePath + 'repaired_NHTS.csv')

### 3) Trip mode

In [14]:
savePath = 'E:/data/Travel_survey/NHTS/NHTS_data/'

trippub_total = pd.read_csv(savePath + 'trippub_total.csv')
trippub_wiscon = pd.read_csv(savePath + 'trippub_wiscon.csv')

In [16]:
trippup_tripMode = preprocess_NHTS_tripMode(trippub_total)

0) Mapping trippurpose:   0%|          | 0/905600 [00:00<?, ?it/s]

1) Compute probability of trip mode choice by age and trip purpose ... 


In [17]:
trippup_tripMode

,age_class,Trip_pur,Trip_mode,nominator,denominator,Trip_modeP
0,Adult,D_shop,Bicy,126,15158,0.008312
1,Adult,D_shop,Car,13528,15158,0.892466
2,Adult,D_shop,PTrans,210,15158,0.013854
3,Adult,D_shop,Walk,1294,15158,0.085367
4,Adult,Home,Bicy,727,46155,0.015751
...,...,...,...,...,...,...
172,Teen,V_fr_rel,Walk,464,2856,0.162465
173,Teen,Work,Bicy,29,1856,0.015625
174,Teen,Work,Car,1674,1856,0.901940
175,Teen,Work,PTrans,34,1856,0.018319


In [ ]:
path = 'E:/data/Chapter_3_data/Analysis/1_Preprocessed_NHTS_data/'
# trippup_tripMode.to_csv(path + 'trip_mode_prop_all.csv')